In [2]:
import rasterio
from rasterio.enums import Resampling
from rasterio.warp import calculate_default_transform, reproject

In [5]:
# Input and output file paths
input_file = "/home/thpc/workspace/pfas_map_ml/data/summa/HUC8_CA_PFAS_GTruth_Summa.tif"
output_file = "/home/thpc/workspace/pfas_map_ml/data/summa/HUC8_CA_PFAS_GTruth_Summa3.tiff"

# Desired resolution in pixels
new_width = 10215  # New width in pixels
new_height = 9469  # New height in pixels

with rasterio.open(input_file) as src:
    # Calculate new pixel size
    x_res = (src.bounds.right - src.bounds.left) / new_width
    y_res = (src.bounds.top - src.bounds.bottom) / new_height

    # Create a new transform with the updated pixel size
    new_transform = rasterio.transform.from_origin(
        src.bounds.left, src.bounds.top, x_res, y_res
    )

    # Write the raster with new dimensions
    with rasterio.open(
        output_file, 'w',
        driver='GTiff',
        height=new_height,
        width=new_width,
        count=src.count,
        dtype=src.dtypes[0],
        crs=src.crs,
        transform=new_transform,
    ) as dst:
        # Reproject and write each band
        for i in range(1, src.count + 1):
            reproject(
                source=rasterio.band(src, i),
                destination=rasterio.band(dst, i),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=new_transform,
                dst_crs=src.crs,
                resampling=Resampling.bilinear,
            )

print(f"Resampled raster saved to {output_file}")


Resampled raster saved to /home/thpc/workspace/pfas_map_ml/data/summa/HUC8_CA_PFAS_GTruth_Summa3.tiff
